In [1]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0
Q.<rad5> = NumberField(xx^2-5)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [4]:
# Define cubic SE10, find and classify the lines and compute the 45 tritangent planes

In [5]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE10 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : (2-rad5)*c})

In [6]:
# Find simmetries

In [7]:
%time adm_SE10 = SE10.find_admissible_projectivities()
len(adm_SE10)

CPU times: user 954 ms, sys: 60.7 ms, total: 1.01 s
Wall time: 7.67 s


120

In [8]:
%time simm_SE10 = SE10.find_simmetries(adm_SE10)
len(simm_SE10)

CPU times: user 627 ms, sys: 104 ms, total: 731 ms
Wall time: 1.65 s


120

In [9]:
# Study how the simmetries permute the Eckardt points

In [10]:
Eck_perms = [Permutation(SE10.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE10]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(120, 'S5')

In [11]:
### geometric interpretation of the group of symmetries

In [13]:
## Action of the stabilizer of SE10 on the 27 lines:
Ga = perm_group_lines(SE10, simm_SE10)

## Action of the stabilizer of SE10 on the 45 tritangent planes:
Gb = perm_group_planes(SE10, simm_SE10)

print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
S5

Structure of the group Gb:
S5


In [14]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [12]:
# We want to find what elements are permuted by the group S5
# These are the so called Sylvester planes

In [19]:
f = SymmetricGroup(5).isomorphism_to(Eck_perms_group)
gens = [f(gen) for gen in SymmetricGroup(5).gens_small()]
indices = [Eck_perms.index(gen) for gen in gens]
gen1 = simm_SE10[indices[0]][0]
gen2 = simm_SE10[indices[1]][0]

In [20]:
Sylvester_planes_labels = [['E1', 'G3', 'F13'], ['E3', 'G4', 'F34'], ['E2', 'G1', 'F12'], ['E4', 'G2', 'F24'], ['F14', 'F23', 'F56']]

In [21]:
bool([label in SE10.eckardt_points_labels for label in Sylvester_planes_labels])

True

In [22]:
#Readjust the coefficients of the planes to match those of the cubic
Sylvester_planes = [pl.plane/min(pl.plane.coefficients()) for pl in SE10.tritangent_planes if pl.labels in Sylvester_planes_labels]
for i in range(len(Sylvester_planes)):
    if i in [0,1]:
        Sylvester_planes[i]*=-1
    if i in [2, 4]:
        Sylvester_planes[i]*=2

In [23]:
# Check that the sum of the planes cubed is actually SE10
mon = ((x+y+z+t)^3).monomials()
nc = sum([pl^3 for pl in Sylvester_planes])
[el for el in matrix([[nc.coefficient(mn) for mn in mon], [SE10.eqn.coefficient(mn) for mn in mon]]).minors(2) if el!=0]

[]

In [24]:
sost1 = change_coordinates(gen1)
sost2 = change_coordinates(gen2)

In [25]:
[True in [pl.subs(sost1).divides(pl1) for pl in Sylvester_planes] for pl1 in Sylvester_planes]

[True, True, True, True, True]

In [26]:
[True in [pl.subs(sost2).divides(pl1) for pl in Sylvester_planes] for pl1 in Sylvester_planes]

[True, True, True, True, True]

In [27]:
# Study how the simmetries permute the 27 lines

In [28]:
lines_perms = [Permutation(SE10.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE10]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(120, 'S5')

In [29]:
gens_lines = [lines_perms[index] for index in indices]
[get_permuted_extended_L_set(perm) for perm in gens_lines], gens_lines

([('G1', 'F14', 'G4', 'E2', 'F56', 'F26'),
  ('F12', 'F34', 'G3', 'E2', 'E3', 'F25')],
 [(1,7,22,18,19)(2,10,15,4,9)(3,27,8,14,13)(5,21,26,24,11)(6,20,25,23,12),
  (1,13)(2,9)(4,19)(5,20)(6,21)(7,14)(10,22)(11,23)(12,24)(15,27)(16,26)(17,25)])